# 11-mer
The goal of this notebook is to reproduce the 11-mer model.
While investigating the plotting notebook in the original repository it was found that the 11-mer model actually is the best markov model. In the config file for the best markov model in the results folder it can be seen, that it is a bidirectional markov model of order 5.


In [26]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import re

import numpy as np

from markov_model import *

import pickle

from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
markov_order = 5

In [27]:
data_dir = '/s/project/mll/sergey/effect_prediction/MLM/motif_predictions/split_75_25/'

In [28]:
motifs = pd.read_csv(data_dir + '../motifs.csv').motif.unique()

In [29]:
#see select_data.ipynb

train_df = pd.read_csv(data_dir + 'train.csv')
test_df = pd.read_csv(data_dir + 'test.csv')

In [30]:
# get the frequency counts of all motifs till 11mer
kmer_train = KmerCount(11,pseudocount=0.1)

kmer_train.compute_counts(train_df.seq)

100%|██████████| 13600/13600 [03:28<00:00, 65.30it/s]


In [31]:
markov_model = BiMarkov(kmer_train)

In [32]:
# calculate the markov matrix using the 11mer counts
markov_model.compile_from_counts()

/data/nasif12/home_if12/l_vilov/workspace/species-aware-DNA-LM/mammals_model/motif_predictions/11_mer/markov_model.py:150: RuntimeWarning: invalid value encountered in divide
  self.markov_matrix[order,:,:] = self.markov_matrix[order,:,:]/np.sum(self.markov_matrix[order,:,:],axis=1)[:,np.newaxis]


In [33]:
test_df = test_df.set_index('seq_name').seq #dataframe to series

In [43]:
#inference

mapping = {'A':0,'C':1,'G':2,'T':3}

motif_probas = []

for seq_name, seq in tqdm(test_df.items(),total = len(test_df)):
    
    prbs = markov_model.impute_for_seq(seq, order=markov_order)
    
    seq_mapping = [mapping[base] for base in seq] #motif sequence to ordinal encoding
    
    target_probas = [prob[seq_mapping[idx]] for idx, prob in enumerate(prbs)] #take the right base
    
    motif_probas.append((seq_name, target_probas))
        
    #for motif in motifs:
    #    
    #    motif_mapping = [mapping[base] for base in motif] #motif sequence to ordinal encoding
    #
    #    for match in re.finditer(motif, seq):
    #
    #        target_probas = prbs[match.start():match.end()] #probabilities for ACGT at each motif position
    #
    #        target_probas = [prob[motif_mapping[idx]] for idx, prob in enumerate(target_probas)] #take the right base 
    #
    #        avg_target_probas = np.mean(target_probas)
    #        
    #        motif_probas.append((seq_name, motif,match.start(),avg_target_probas))

100%|██████████| 4534/4534 [00:26<00:00, 169.75it/s]


In [36]:
#motif_probas = pd.DataFrame(motif_probas, columns=['seq_name','motif','motif_start','avg_target_prob'])

#motif_probas.to_csv(data_dir + '11_mer.tsv', sep='\t', index=None)

In [47]:
with open(data_dir + '11_mer.pickle', 'wb') as f:
    pickle.dump(motif_probas,f)